In [13]:
from llama_index import SimpleDirectoryReader
from llama_index import VectorStoreIndex
from llama_index import LLMPredictor, ServiceContext
from llama_index import StorageContext, load_index_from_storage
from llama_index import PromptHelper
from llama_index import Prompt
from llama_index.llms import OpenAI
# from langchain.chat_models import ChatOpenAI
import openai
import environ
from IPython.display import Markdown, display

env = environ.Env()
environ.Env.read_env()
API_KEY = env('OPENAI_API_KEY')
openai.api_key = API_KEY

/home/daniele/Desktop/Projects/chatbot_dsr_project/.venv_chatbot/lib/python3.10/site-packages/environ/environ.py:639: UserWarning: /tmp/ipykernel_15151/.env doesn't exist - if you're not configuring your environment separately, create one.
  warnings.warn(


In [2]:
doc_path = "documents_pdf"
embedding_path = "vector_db"

# Create and save vector store

In [3]:
# Configure prompt parameters and initialise helper
max_input_size = 4096
num_output = 256
max_chunk_overlap = 0.2
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [4]:
docs = SimpleDirectoryReader(input_dir=doc_path).load_data()

# system_prompt = (
#     """
#     You are an expert in the German administrative system, and your job is to answer technical questions.
#     Assume that all questions are related to the provided context.
#     Keep your answers based on facts, do not hallucinate information.
#     """
# )

# llm = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
# llm= LLMPredictor( llm=OpenAI(temperature=0, model_name="text-davinci-003"))
llm= LLMPredictor(llm=OpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
    #system_prompt=system_prompt
))

storage_context = StorageContext.from_defaults()
service_context = ServiceContext.from_defaults(llm_predictor=llm, prompt_helper=prompt_helper)
index = VectorStoreIndex.from_documents(
    docs,
    service_context=service_context,
    storage_context=storage_context
)

In [5]:
# The follwoing 2 lines should save the save the same vector store
# storage_context.persist(persist_dir=embedding_path)
index.storage_context.persist(persist_dir=embedding_path)

# Load

In [6]:
# rebuild storage context
storage_context_from_load = StorageContext.from_defaults(persist_dir=embedding_path)
# load index
index_loaded = load_index_from_storage(storage_context_from_load)

In [7]:
# Define prompt
template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Do not give me an answer if it is not mentioned in the context as a fact. \n"
    "Given this information, please provide me with an answer to the following question:\n{query_str}\n"
)
qa_template = Prompt(template)

query_engine_loaded = index_loaded.as_query_engine(text_qa_template=qa_template, similarity_top_k=3,)

In [17]:
response = query_engine_loaded.query("What is the capital of Italy?")

In [18]:
display(Markdown(response.response))

The context information provided does not mention the capital of Italy.

In [10]:
response = query_engine_loaded.query("Is the unemployment benefit based on the number of childer I have?")

In [12]:
print(response.response)

Based on the provided context, the unemployment benefit is not directly based on the number of children an individual has. However, if an individual has at least one child, the unemployment benefit can be increased to 67% of the pauschalierten Nettoentgelt (net income) instead of the standard 60%. This increase in benefit is applicable if certain conditions are met.


In [15]:
display(Markdown(response.response))

Based on the provided context, the unemployment benefit is not directly based on the number of children an individual has. However, if an individual has at least one child, the unemployment benefit can be increased to 67% of the pauschalierten Nettoentgelt (net income) instead of the standard 60%. This increase in benefit is applicable if certain conditions are met.